In [12]:
import duckdb
import csv

conn = duckdb.connect()
conn = duckdb.connect(database='../db.duckdb', read_only=False)

In [20]:
# Data dir
geoname_base_dir: str = "../data"
geoname_csv_dir: str = f"{geoname_base_dir}/csv"
geoname_pqt_dir: str = f"{geoname_base_dir}/parquet"


In [26]:
from pathlib import Path

def searching_all_files(directory: Path):   
    file_list = [] # A list for storing files existing in directories

    for x in directory.iterdir():
        if x.is_file():

           file_list.append(x)
        else:

           file_list.append(searching_all_files(directory/x))

    return file_list


print(searching_all_files(Path(geoname_base_dir)))


[[PosixPath('../data/../data/csv/.gitkeep'), PosixPath('../data/../data/csv/allCountries.txt'), PosixPath('../data/../data/csv/alternateNames.txt')], [PosixPath('../data/../data/parquet/.gitkeep'), PosixPath('../data/../data/parquet/geonames.parquet'), PosixPath('../data/../data/parquet/alternateNames.parquet'), PosixPath('../data/../data/parquet/allCountries.parquet')]]


In [27]:
# allCountries
geoname_allctry_fn: str = "allCountries"
geoname_allctry_csv: str = f"{geoname_csv_dir}/{geoname_allctry_fn}.txt"
geoname_allctry_pqt: str = f"{geoname_pqt_dir}/{geoname_allctry_fn}.parquet"

# Alternate names
geoname_altname_fn: str = "alternateNames"
geoname_altname_csv: str = f"{geoname_csv_dir}/{geoname_altname_fn}.txt"
geoname_altname_pqt: str = f"{geoname_pqt_dir}/{geoname_altname_fn}.parquet"

#
geoame_nce_query: str = "select * from geonames where isoLanguage='iata' and alternateName='NCE'"

def countRows():
    """
    Check that everything goes right
    """
    count_query: str = """
    select count(*)/1e6 as nb from allcountries
    union all
    select count(*)/1e6 as nb from altnames
    union all
    select count(*)/1e6 as nb from geonames
    """

    nb_list = conn.execute(count_query).fetchall()
    return nb_list

def getNCErows():
    """
    Retrieve all the records featuring NCE as the IATA code
    """
    nce_recs = conn.execute(geoame_nce_query).fetchall()
    return nce_recs

In [28]:
nb_list = countRows()
print(f"Number of rows: {nb_list}")


IOException: IO Error: No files found that match the pattern "data/parquet/allCountries.parquet"

In [17]:
nce_recs = getNCErows()
print("List of records featuring NCE as the IATA code:")
for nce_rec in nce_recs:
    print(nce_rec)

IOException: IO Error: No files found that match the pattern "data/parquet/allCountries.parquet"